### Adjust Word Group In Youtube Sentence Word Usage Analysis

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# word all usage in sent
word_use_num_max = 5
word_use_num_min = 1

# youtube sentence usage for remain word 
youtube_sent_usage = "selected"  # "all","selected"

In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/3-Adjust Word Group In Youtube Sentence Word Usage Analysis/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num):

    '''word_in_wordgroup_simple(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number. 
    '''
    
    df_select = df_target[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Word Usage

In [6]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [7]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [8]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,bir,18835735
1,1,bu,11062659
2,2,ne,8025880
3,3,ve,7766036
4,4,için,5484109
...,...,...,...
195,195,saat,399989
196,196,onunla,399330
197,197,yapıyorsun,398274
198,198,neler,397377


In [9]:
df_youtube_sent_result = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment2/Result/\
2-Word Group In Youtube Sentence/{lang_folder.lower().capitalize()}/{lang_folder.lower().capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx")
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...
1,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...
3,çok güzel bir dakika bir dakika bir dakika bir...,525,531,çok güzel bir dakika bir dakika bir dakika bir...,OZhgDD1ScBg,https://www.youtube.com/watch?v=OZhgDD1ScBg&t=...
4,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
...,...,...,...,...,...,...
187595,o da o da,7102,7103,ha o da o da iptal peki peki,8DYpvzyEVg0,NaN
187596,ol o o bu,244,250,abone ol o o bu çocuğumuz sünnet ettirme ne iz...,oixYiG0tGgo,NaN
187597,o ne o o,3947,3948,o ne o o i lyasın yanındaki ayşen mi ya,D6Va3TXwFEQ,NaN
187598,ol o o o,2258,2260,abone ol o o o,K0b_9Xzy79A,NaN


In [10]:
df_youtube_sent_result["word_index_list"] = df_youtube_sent_result["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
1,yardım yapıyorsun çok teşekkürler çok zaman he...,1222,1230,yardım yapıyorsun çok teşekkürler çok zaman he...,zgQbOmPp9pk,https://www.youtube.com/watch?v=zgQbOmPp9pk&t=...,"[87, 197, 9, 100, 9, 35, 30, 35, 60, 30, 35, 106]"
2,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[45, 56, 0, 16, 129, 5, 45, 10, 0, 16, 129, 5,..."
3,çok güzel bir dakika bir dakika bir dakika bir...,525,531,çok güzel bir dakika bir dakika bir dakika bir...,OZhgDD1ScBg,https://www.youtube.com/watch?v=OZhgDD1ScBg&t=...,"[9, 54, 0, 177, 0, 177, 0, 177, 0, 177, 0, 177..."
4,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s,"[36, 4, 156, 79, 72, 16, 167, 38, 69, 1, 132, ..."
...,...,...,...,...,...,...,...
187595,o da o da,7102,7103,ha o da o da iptal peki peki,8DYpvzyEVg0,NaN,"[6, 15, 6, 15]"
187596,ol o o bu,244,250,abone ol o o bu çocuğumuz sünnet ettirme ne iz...,oixYiG0tGgo,NaN,"[118, 6, 6, 1]"
187597,o ne o o,3947,3948,o ne o o i lyasın yanındaki ayşen mi ya,D6Va3TXwFEQ,NaN,"[6, 2, 6, 6]"
187598,ol o o o,2258,2260,abone ol o o o,K0b_9Xzy79A,NaN,"[118, 6, 6, 6]"


In [11]:
#df_youtube_sent_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List.xlsx", index=False)

In [12]:
df_youtube_sent_select = df_youtube_sent_result[df_youtube_sent_result["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
1,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[45, 56, 0, 16, 129, 5, 45, 10, 0, 16, 129, 5,..."
2,çok güzel bir dakika bir dakika bir dakika bir...,525,531,çok güzel bir dakika bir dakika bir dakika bir...,OZhgDD1ScBg,https://www.youtube.com/watch?v=OZhgDD1ScBg&t=...,"[9, 54, 0, 177, 0, 177, 0, 177, 0, 177, 0, 177..."
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s,"[36, 4, 156, 79, 72, 16, 167, 38, 69, 1, 132, ..."
4,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...,"[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]"
...,...,...,...,...,...,...,...
56663,dur o o o,118,120,tür ve alt türler uyguna giderseniz kalan şu b...,tmHOrGYUtEE,NaN,"[144, 6, 6, 6]"
56664,ki ve o o,421,422,kızım o kadar şaşırıyor ki ve o o anda diyor ki,yZ0ijoRJJiw,NaN,"[33, 3, 6, 6]"
56665,o ne o ne,188,190,o ne o ne,uBDsQXn1DgQ,NaN,"[6, 2, 6, 2]"
56666,ol o o bu,244,250,abone ol o o bu çocuğumuz sünnet ettirme ne iz...,oixYiG0tGgo,NaN,"[118, 6, 6, 1]"


In [13]:
# # word usage in max condition
# result_list = []
# index_list = []
# for i in range(len(df_youtube_sent_select)):
#     search = df_youtube_sent_select.loc[i,"search_string"]
#     start=df_youtube_sent_select.loc[i,"start_time"]
#     end=df_youtube_sent_select.loc[i,"end_time"]
#     sent=df_youtube_sent_select.loc[i,"sentence"]
#     id = df_youtube_sent_select.loc[i,"video_id"]
#     id_url = df_youtube_sent_select.loc[i,"video_url"]
#     word_index = df_youtube_sent_select.loc[i,"word_index_list"]
#     list_var = index_list + word_index
#     #list_var = index_list + [x for x in word_index]
# 
#     dict_list_count = Counter(list_var)
#     count_list = list(dict_list_count.values())
# 
#     if any([True if i>word_use_num_max else False for i in count_list]):
#         index_list = index_list
#     else:
#         index_list = index_list + word_index
#         result_list.append([search,start,end,sent,id,id_url,word_index])
#     

In [14]:
# word usage in min max condition
word_num_index_dict = {}
for i in df_word_select["index"]:
    word_num_index_dict[i] = 0

result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + word_index
    #list_var = index_list + [x for x in word_index]
    # word count for max
    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())
     # word count for min
    count_list2 = list(word_num_index_dict.values())   

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])

        for item2 in dict_list_count.items(): 
            word_num_index_dict[item2[0]] = item2[1]

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}") 

Max condition: False 
Min condition: True


In [15]:
#word_num_index_dict

In [16]:
set_index = set(index_list)
set_all_index = set([x for x in df_word_select["index"]])

In [17]:
not_in_sent = set_all_index.difference(set_index)
not_in_sent = list(not_in_sent)
not_in_sent

[162, 100, 164, 137, 171, 141, 81, 117, 118, 149, 151, 185, 122, 92, 157]

In [18]:
#Counter(index_list)

In [19]:
df_youtube_sent_sample = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index"])
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
1,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[45, 56, 0, 16, 129, 5, 45, 10, 0, 16, 129, 5,..."
2,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...,"[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]"
3,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
4,şeyler olabilir ne gibi senin gerçekten ben de...,378,383,şeyler olabilir ne gibi senin gerçekten ben de...,YdTPitK_hV0,https://www.youtube.com/watch?v=YdTPitK_hV0&t=...,"[107, 129, 2, 27, 36, 70, 7, 8, 94, 10]"
...,...,...,...,...,...,...,...
116,doğru bak onu doğru,577,579,doğru bak onu doğru dedi,3PDJDym1ZXQ,NaN,"[75, 64, 23, 75]"
117,bile eğer bizim ona,235,238,masum en kullanımı kolay ürün bile eğer bizim ...,un1S350T4qU,NaN,"[59, 67, 148, 57]"
118,diye söyle bak hala,104,106,o taşlarını gecesi zor için yoğurduktan sonra ...,P5bYtFrqw68,NaN,"[123, 161, 64, 167]"
119,efendim al al al al,497,499,ben hallediyorum efendim al al al al,jNTZDRtKZqs,NaN,"[85, 169, 169, 169, 169]"


In [20]:
df_youtube_sent_sample.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx", index=False)

##### Selected Result Word Count

In [21]:
#df_selected_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/Deploy Result Manuel/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised.xlsx")
df_selected_file = pd.read_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample.xlsx")
df_selected_file

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...,"[94, 74, 84, 16, 180, 74, 9, 16, 80, 9, 16, 80..."
1,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...,"[45, 56, 0, 16, 129, 5, 45, 10, 0, 16, 129, 5,..."
2,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...,"[6, 9, 54, 55, 70, 173, 27, 55, 70, 12, 43]"
3,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
4,şeyler olabilir ne gibi senin gerçekten ben de...,378,383,şeyler olabilir ne gibi senin gerçekten ben de...,YdTPitK_hV0,https://www.youtube.com/watch?v=YdTPitK_hV0&t=...,"[107, 129, 2, 27, 36, 70, 7, 8, 94, 10]"
...,...,...,...,...,...,...,...
116,doğru bak onu doğru,577,579,doğru bak onu doğru dedi,3PDJDym1ZXQ,NaN,"[75, 64, 23, 75]"
117,bile eğer bizim ona,235,238,masum en kullanımı kolay ürün bile eğer bizim ...,un1S350T4qU,NaN,"[59, 67, 148, 57]"
118,diye söyle bak hala,104,106,o taşlarını gecesi zor için yoğurduktan sonra ...,P5bYtFrqw68,NaN,"[123, 161, 64, 167]"
119,efendim al al al al,497,499,ben hallediyorum efendim al al al al,jNTZDRtKZqs,NaN,"[85, 169, 169, 169, 169]"


In [22]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [23]:
df_shared_word_manuel_count = word_count_result(df_selected_file, df_selected_file.columns[0:1])
df_shared_word_manuel_count

,word,word_count
0,al,5
1,gibi,5
2,olarak,5
3,olan,5
4,olacak,5
...,...,...
180,kötü,1
181,selam,1
182,vardı,1
183,sorun,1


In [24]:
df_shared_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_Word_Count.xlsx", index=False)

##### Not In Sent And Selected Word List

In [25]:
lang_folder = "Turkish"
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [26]:
word_start = 0
word_end = 200
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [27]:
def index_to_word(index_list, df_word, word_column, reverse=True):
    '''
    index_to_word(not_in_sent, df_word_select, "word", reverse=True)\n
    not in sent is index list as numeric\n
    df_word_select is a dataframe that includes word colunm ("word" is a column)\n
    reverse is used for string index list convert numeric.
    '''
    if reverse:
        word_index_dict = {var1:var2 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    else:
        word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}

    convert_word_list = []
    for index in index_list:
        convert_word = word_index_dict[index]
        convert_word_list.append(convert_word)
    return convert_word_list        

In [28]:
not_in_sent_word = index_to_word(not_in_sent, df_word_select, "word", reverse=True)
not_in_sent_word

['özür',
 'teşekkürler',
 'dostum',
 'haydi',
 'üç',
 'sizi',
 'adam',
 'gerek',
 'ol',
 'asla',
 'mısın',
 'pekala',
 'üzgünüm',
 'tüm',
 'benimle']

In [29]:
set_not_in_sent_word = set(not_in_sent_word)

In [30]:
set_word = set(df_word_select["word"])
set_select_word = set(df_shared_word_manuel_count["word"])

In [31]:
set_not_in_select = set_word.difference(set_select_word)
set_not_in_select

{'adam',
 'asla',
 'benimle',
 'dostum',
 'gerek',
 'haydi',
 'mısın',
 'ol',
 'pekala',
 'sizi',
 'teşekkürler',
 'tüm',
 'özür',
 'üzgünüm',
 'üç'}

In [32]:
not_in_sent_selected_word_list = list(set_not_in_select.union(set_not_in_sent_word))
not_in_sent_selected_word_list

['ol',
 'teşekkürler',
 'sizi',
 'dostum',
 'asla',
 'tüm',
 'özür',
 'pekala',
 'mısın',
 'benimle',
 'üzgünüm',
 'adam',
 'haydi',
 'üç',
 'gerek']

In [33]:
len(not_in_sent_selected_word_list)

15

In [34]:
df_youtube_sent_select.drop(["word_index_list"], axis=1, inplace=True)
df_youtube_sent_select

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id,video_url
0,bilmiyorum belki hiçbir şey olmaz belki çok şe...,381,385,bilmiyorum belki hiçbir şey olmaz belki çok şe...,L3bf7dlR5hk,https://www.youtube.com/watch?v=L3bf7dlR5hk&t=...
1,ya böyle bir şey olabilir mi ya ama bir şey ol...,466,480,ya böyle bir şey olabilir mi ya ama bir şey ol...,duaQwKTmgvE,https://www.youtube.com/watch?v=duaQwKTmgvE&t=...
2,çok güzel bir dakika bir dakika bir dakika bir...,525,531,çok güzel bir dakika bir dakika bir dakika bir...,OZhgDD1ScBg,https://www.youtube.com/watch?v=OZhgDD1ScBg&t=...
3,senin için önemli olan tek şey hala burada bil...,84,88,senin için önemli olan tek şey hala burada bil...,9Wx3JJTq_Hw,https://www.youtube.com/watch?v=9Wx3JJTq_Hw&t=84s
4,o çok güzel oldu gerçekten herkes gibi oldu ge...,394,401,o çok güzel oldu gerçekten herkes gibi oldu ge...,f8iA6v4mRAY,https://www.youtube.com/watch?v=f8iA6v4mRAY&t=...
...,...,...,...,...,...,...
56663,dur o o o,118,120,tür ve alt türler uyguna giderseniz kalan şu b...,tmHOrGYUtEE,NaN
56664,ki ve o o,421,422,kızım o kadar şaşırıyor ki ve o o anda diyor ki,yZ0ijoRJJiw,NaN
56665,o ne o ne,188,190,o ne o ne,uBDsQXn1DgQ,NaN
56666,ol o o bu,244,250,abone ol o o bu çocuğumuz sünnet ettirme ne iz...,oixYiG0tGgo,NaN


In [35]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [36]:
if youtube_sent_usage == "all": 
    df_youtube_usage_select = df_youtube_sent_result
else:
    df_youtube_usage_select = df_youtube_sent_select

In [37]:
not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_selected_word_list, df_youtube_usage_select, "search_string", 5, simple=False)
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,ol,ol ama çok değil değil mi biraz var ama evet ş...,406,415,abone ol ama çok değil değil mi biraz var ama ...,o4LyycKz0wg,https://www.youtube.com/watch?v=o4LyycKz0wg&t=...
1,ol,ol çok teşekkür ederim çok teşekkür ederim,494,497,sağ ol çok teşekkür ederim çok teşekkür ederim,kolVH5PDzI0,https://www.youtube.com/watch?v=kolVH5PDzI0&t=...
2,ol,tamam tamam adam ol adam ol adam ol böyle,383,389,kesin anlayacak video olduğunu gitmiş ha tamam...,qnZMqt9UQo0,https://www.youtube.com/watch?v=qnZMqt9UQo0&t=...
3,ol,ol o çok güzel çok teşekkür ederim sen de,442,447,abone ol o çok güzel çok teşekkür ederim sen d...,OPWhkKQLTmE,https://www.youtube.com/watch?v=OPWhkKQLTmE&t=...
4,ol,ol evet şimdi tekrar yine aynı şekilde,613,618,abone ol evet şimdi tekrar yine aynı şekilde p...,pw2ei7jBsCg,https://www.youtube.com/watch?v=pw2ei7jBsCg&t=...
...,...,...,...,...,...,...,...
67,gerek,için artık değil misin gerek yok bunun için si...,65,70,göndermek için artık değil misin gerek yok bun...,gX77n13Qj0o,https://www.youtube.com/watch?v=gX77n13Qj0o&t=65s
68,gerek,ne demek ne gerek ne demek gerek değil mi nası...,648,653,çok doğru konuşuyorsun doğru söze ne demek ne ...,IzlEfOAiIw8,https://www.youtube.com/watch?v=IzlEfOAiIw8&t=...
69,gerek,gerek yok çok teşekkür ederim ben zaten birkaç,186,190,gerek yok çok teşekkür ederim ben zaten birkaç...,zpfhmsZB26w,https://www.youtube.com/watch?v=zpfhmsZB26w&t=...
70,gerek,gerek yok evet şimdi benim için önemli olan,550,554,program kurmaya gerek yok evet şimdi benim içi...,9NguZ1ukR5E,https://www.youtube.com/watch?v=9NguZ1ukR5E&t=...


In [38]:
#not_in_sent_selected_word_list_merge = pd.merge(not_in_sent_selected_word_list_result, df_youtube_sent_select, how="left", on="search_string")
#not_in_sent_selected_word_list_merge.drop_duplicates(inplace=True)
#not_in_sent_selected_word_list_merge

In [39]:
not_in_sent_selected_word_list_result.word.nunique()

15

In [40]:
set_not_in_sent_selected_word_list = set(not_in_sent_selected_word_list)
set_not_in_sent_selected_word_list_result = set(not_in_sent_selected_word_list_result["word"].to_list())

In [41]:
set_not_in_sent_selected_word_list.difference(set_not_in_sent_selected_word_list_result)

set()

In [42]:
#word_in_wordgroup_simple(["pekala","gece"], df_youtube_sent_select, "search_string", 5)

In [43]:
not_in_sent_selected_word_list_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample2.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_*.xlsx")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass